In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import shuffle


# >> FEATURE SELECTION << #
def remove_correlated_features(X):
    corr_threshold = 0.9
    corr = X.corr()
    drop_columns = np.full(corr.shape[0], False, dtype=bool)
    for i in range(corr.shape[0]):
        for j in range(i + 1, corr.shape[0]):
            if corr.iloc[i, j] >= corr_threshold:
                drop_columns[j] = True
    columns_dropped = X.columns[drop_columns]
    X.drop(columns_dropped, axis=1, inplace=True)
    return columns_dropped


def remove_less_significant_features(X, Y):
    sl = 0.05
    regression_ols = None
    columns_dropped = np.array([])
    for itr in range(0, len(X.columns)):
        regression_ols = sm.OLS(Y, X).fit()
        max_col = regression_ols.pvalues.idxmax()
        max_val = regression_ols.pvalues.max()
        if max_val > sl:
            X.drop(max_col, axis='columns', inplace=True)
            columns_dropped = np.append(columns_dropped, [max_col])
        else:
            break
    regression_ols.summary()
    return columns_dropped


##############################


# >> MODEL TRAINING << #
def compute_cost(W, X, Y):
    # calculate hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss = regularization_strength * (np.sum(distances) / N)

    # calculate cost
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost


# I haven't tested it but this same function should work for
# vanilla and mini-batch gradient descent as well
def calculate_cost_gradient(W, X_batch, Y_batch):
    # if only one example is passed (eg. in case of SGD)
    if type(Y_batch) == np.float64:
        Y_batch = np.array([Y_batch])
        X_batch = np.array([X_batch])  # gives multidimensional array

    distance = 1 - (Y_batch * np.dot(X_batch, W))
    dw = np.zeros(len(W))

    for ind, d in enumerate(distance):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (regularization_strength * Y_batch[ind] * X_batch[ind])
        dw += di

    dw = dw/len(Y_batch)  # average
    return dw


def sgd(features, outputs):
    max_epochs = 2
    weights = np.zeros(features.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 0.01  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)

        # convergence check on 2^nth epoch
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = compute_cost(weights, features, outputs)
            print("Epoch is: {} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1
    return weights


########################


def init():
    print("reading dataset...")
    # read data in pandas (pd) data frame
    data = pd.read_csv('./data/data.csv')

    # drop last column (extra column added by pd)
    # and unnecessary first column (id)
    data.drop(data.columns[[-1, 0]], axis=1, inplace=True)

    print("applying feature engineering...")
    # convert categorical labels to numbers
    diag_map = {'M': 1.0, 'B': -1.0}
    data['diagnosis'] = data['diagnosis'].map(diag_map)

    # put features & outputs in different data frames
    Y = data.loc[:, 'diagnosis']
    X = data.iloc[:, 1:]

    # filter features
    remove_correlated_features(X)
    remove_less_significant_features(X, Y)

    # normalize data for better convergence and to prevent overflow
    X_normalized = MinMaxScaler().fit_transform(X.values)
    X = pd.DataFrame(X_normalized)

    # insert 1 in every row for intercept b
    X.insert(loc=len(X.columns), column='intercept', value=1)

    # split data into train and test set
    print("splitting dataset into train and test sets...")
    X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.2, random_state=42)

    # train the model
    print("training started...")
    W = sgd(X_train.to_numpy(), y_train.to_numpy())
    print("training finished.")
    print("weights are: {}".format(W))

    # testing the model
    print("testing the model...")
    y_train_predicted = np.array([])
    for i in range(X_train.shape[0]):
        yp = np.sign(np.dot(X_train.to_numpy()[i], W))
        y_train_predicted = np.append(y_train_predicted, yp)

    y_test_predicted = np.array([])
    for i in range(X_test.shape[0]):
        yp = np.sign(np.dot(X_test.to_numpy()[i], W))
        y_test_predicted = np.append(y_test_predicted, yp)

    print("accuracy on test dataset: {}".format(accuracy_score(y_test, y_test_predicted)))
    print("recall on test dataset: {}".format(recall_score(y_test, y_test_predicted)))
    print("precision on test dataset: {}".format(recall_score(y_test, y_test_predicted)))


# set hyper-parameters and call init
regularization_strength = 10000
learning_rate = 0.000001
init()

reading dataset...
applying feature engineering...
splitting dataset into train and test sets...
training started...
Epoch is: 1 and Cost is: 7236.957044832009
training finished.
weights are: [-2.32335007e-02  3.47023567e-01 -1.10514113e-01 -2.09428998e-01
  1.40125991e-01  4.10282452e-03  1.54608912e-04 -2.45955006e-03
 -5.95517495e-02 -5.19184886e-02  2.97345777e-03  2.68810472e-03
 -8.79752540e-01]
testing the model...
accuracy on test dataset: 0.6228070175438597
recall on test dataset: 0.0
precision on test dataset: 0.0


In [48]:
data = pd.read_csv('data.csv')

# drop last column (extra column added by pd)
# and unnecessary first column (id)
data.drop(data.columns[[-1, 0]], axis=1, inplace=True)

print("applying feature engineering...")
# convert categorical labels to numbers
diag_map = {'M': 1.0, 'B': -1.0}
data['diagnosis'] = data['diagnosis'].map(diag_map)

# put features & outputs in different data frames
Y = data.loc[:, 'diagnosis']
X = data.iloc[:, 1:]

# filter features
remove_correlated_features(X)
remove_less_significant_features(X, Y)

# normalize data for better convergence and to prevent overflow
X_normalized = MinMaxScaler().fit_transform(X.values)
X = pd.DataFrame(X_normalized)

# insert 1 in every row for intercept b
X.insert(loc=len(X.columns), column='intercept', value=1)

# split data into train and test set
print("splitting dataset into train and test sets...")
X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.2, random_state=42)

applying feature engineering...
splitting dataset into train and test sets...


In [6]:
W = sgd(X_train.to_numpy(), y_train.to_numpy())
print("weights are: {}".format(W))

Epoch is: 1 and Cost is: 7315.472491416633
weights are: [-4.35306770e-02  3.45350156e-01 -1.28731898e-01 -2.31821474e-01
  1.37026343e-01 -4.13242941e-03 -7.03199116e-04 -4.12065153e-03
 -6.90538524e-02 -7.36492534e-02 -1.13911560e-02 -9.91293385e-03
 -9.29731256e-01]


In [54]:
features = X_train.to_numpy()
outputs = y_train.to_numpy()

max_epochs = 2
weights = np.zeros(features.shape[1])
nth = 0
prev_cost = float("inf")
cost_threshold = 0.01  # in percent
# stochastic gradient descent
for epoch in range(1, max_epochs):
    # shuffle to prevent repeating update cycles
    X, Y = shuffle(features, outputs)
    for ind, x in enumerate(X):
        ascent = calculate_cost_gradient(weights, x, Y[ind])
        weights = weights - (learning_rate * ascent)

    # convergence check on 2^nth epoch
    if epoch == 2 ** nth or epoch == max_epochs - 1:
        cost = compute_cost(weights, features, outputs)
        print("Epoch is: {} and Cost is: {}".format(epoch, cost))
        # stoppage criterion
        if abs(prev_cost - cost) < cost_threshold * prev_cost:
            weights
        prev_cost = cost
        nth += 1

Epoch is: 1 and Cost is: 7222.079227476714


In [59]:
weights = np.zeros(features.shape[1])
weights

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [55]:
max_epochs = 2
for epoch in range(1, max_epochs):
    # shuffle to prevent repeating update cycles
    X, Y = shuffle(features, outputs)
    
(X[:10], y[:10])

(array([[0.20121745, 0.04889878, 0.26717172, 0.12426285, 0.03718993,
         0.05246793, 0.02326515, 0.20382648, 0.04177549, 0.21283761,
         0.11255667, 0.07910272, 1.        ],
        [0.67974298, 0.10979381, 0.4       , 0.06276327, 0.12913272,
         0.09026046, 0.06285354, 0.17213487, 0.02954549, 0.28811992,
         0.32879953, 0.04335563, 1.        ],
        [0.04058167, 0.06105904, 0.27323232, 0.11478517, 0.05095057,
         0.07351218, 0.03474747, 0.11047547, 0.03324213, 0.4373638 ,
         0.17642421, 0.11530893, 1.        ],
        [0.17314846, 0.06354264, 0.23989899, 0.25652906, 0.02679703,
         0.09341485, 0.0489899 , 0.07895435, 0.09128284, 0.26962953,
         0.20205007, 0.23966942, 1.        ],
        [0.44132567, 0.2628866 , 0.33181818, 0.23188711, 0.07293138,
         0.16664163, 0.11441919, 0.33396477, 0.04308832, 0.36010038,
         0.19534792, 0.08684245, 1.        ],
        [0.11464322, 0.06009841, 0.31212121, 0.44860994, 0.00550425,
         0.

In [15]:
svm_data = pd.DataFrame(X)
svm_data['target'] = Y
svm_data.columns
cols = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 'bias', 'target']
svm_data.columns = cols
weights = np.zeros(X.shape[1])
weights
weights = []
for i in range(len(X)):
    weights.append(0)
svm_data['w1'] = weights
svm_data['w2'] = weights
svm_data['w3'] = weights
svm_data['w4'] = weights
svm_data['w5'] = weights
svm_data['w6'] = weights
svm_data['w7'] = weights
svm_data['w8'] = weights
svm_data['w9'] = weights
svm_data['w10'] = weights
svm_data['w11'] = weights
svm_data['w12'] = weights
svm_data.columns
svm_data

,0,1,2,3,4,5,6,7,8,9,...,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12
0,0.224552,0.023711,0.416667,0.253791,0.035814,0.037515,0.025556,0.104092,0.059119,0.378591,...,0,0,0,0,0,0,0,0,0,0
1,0.365573,0.368322,0.249495,0.140059,0.200181,0.138928,0.080429,0.277704,0.045265,0.433402,...,0,0,0,0,0,0,0,0,0,0
2,0.430504,0.040159,0.244444,0.206403,0.040703,0.073587,0.023763,0.086210,0.051967,0.359440,...,0,0,0,0,0,0,0,0,0,0
3,0.396348,0.248360,0.451010,0.228939,0.192178,0.275768,0.098838,0.344005,0.122652,0.519250,...,0,0,0,0,0,0,0,0,0,0
4,0.291512,0.165651,0.374242,0.320977,0.070433,0.156578,0.074369,0.264823,0.101751,0.644720,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,0.523842,0.056186,0.255051,0.106992,0.399240,0.065251,0.045909,0.363137,0.029684,0.149904,...,0,0,0,0,0,0,0,0,0,0
451,0.105512,0.016806,0.279293,0.214195,0.078255,0.058416,0.016202,0.149555,0.135643,0.153338,...,0,0,0,0,0,0,0,0,0,0
452,0.116672,0.091097,0.151515,0.283909,0.035741,0.110388,0.041717,0.126141,0.063575,0.395760,...,0,0,0,0,0,0,0,0,0,0
453,0.257017,0.047587,0.295455,0.329823,0.062575,0.113393,0.029242,0.180337,0.090903,0.463118,...,0,0,0,0,0,0,0,0,0,0


In [56]:
for ind, x in enumerate(X[:10]):
    print(ind, x)

0 [0.20121745 0.04889878 0.26717172 0.12426285 0.03718993 0.05246793
 0.02326515 0.20382648 0.04177549 0.21283761 0.11255667 0.07910272
 1.        ]
1 [0.67974298 0.10979381 0.4        0.06276327 0.12913272 0.09026046
 0.06285354 0.17213487 0.02954549 0.28811992 0.32879953 0.04335563
 1.        ]
2 [0.04058167 0.06105904 0.27323232 0.11478517 0.05095057 0.07351218
 0.03474747 0.11047547 0.03324213 0.4373638  0.17642421 0.11530893
 1.        ]
3 [0.17314846 0.06354264 0.23989899 0.25652906 0.02679703 0.09341485
 0.0489899  0.07895435 0.09128284 0.26962953 0.20205007 0.23966942
 1.        ]
4 [0.44132567 0.2628866  0.33181818 0.23188711 0.07293138 0.16664163
 0.11441919 0.33396477 0.04308832 0.36010038 0.19534792 0.08684245
 1.        ]
5 [0.11464322 0.06009841 0.31212121 0.44860994 0.00550425 0.11542043
 0.0454798  0.13866263 0.10468748 0.61368289 0.27951902 0.3270366
 1.        ]
6 [0.29861346 0.06192596 0.27373737 0.22535805 0.00304183 0.06172079
 0.0294697  0.10528509 0.01593356 0.31

In [62]:
dw = np.zeros(len(weights))
dw.shape

(13,)

In [68]:
for ind, x in enumerate(X[:3]):
    ascent = calculate_cost_gradient(weights, x, Y[ind])
    #weights = weights - (learning_rate * ascent)
    print(ascent)

[ 2012.17450118   488.98781631  2671.71717172  1242.62847515
   371.89933007   524.67930423   232.65151515  2038.26482288
   417.75493001  2128.37614739  1125.5667258    791.02715466
 10000.        ]
[ -6797.42982753  -1097.93814433  -4000.           -627.63268745
  -1291.32717726   -902.60462042   -628.53535354  -1721.34874029
   -295.45485953  -2881.19923397  -3287.99526907   -433.55634265
 -10000.        ]
[  405.81670612   610.59044049  2732.32323232  1147.85172704
   509.50570342   735.12181933   347.47474747  1104.75468839
   332.42126501  4373.63798455  1764.24206584  1153.08933491
 10000.        ]


In [ ]:
#here ascent is dw

In [82]:
if type(Y[0]) == np.float64:
    Y_batch = np.array([Y[0]])
    X_batch = np.array([X[0]])
    
print(X_batch,Y_batch)

[[0.20121745 0.04889878 0.26717172 0.12426285 0.03718993 0.05246793
  0.02326515 0.20382648 0.04177549 0.21283761 0.11255667 0.07910272
  1.        ]] [-1.]


In [73]:
distance = 1 - (Y_batch * np.dot(X_batch, W))
distance

## distance is the distance from the decision boundary to support vectors

array([-0.00140696])

In [85]:
dw = np.zeros(len(weights))
dw
dw.shape

(13,)

In [75]:
for ind, d in enumerate(distance):
    print(ind,d)

0 -0.0014069629692219365


In [78]:
if max(0,0) == 0:
    print('It is correct')
else:
    print('i')
    
# so if the distance of support vector and decision boundary is 0, or say, our support vector is same as decision boundary we will return previous weights

It is correct


In [91]:
di = weights - (regularization_strength * Y_batch * X_batch)
di
dw =di
dw
# as our initial weights were 0, so dw += di is same as 0+di.. or we can say dw = di.. If you set any weights, that will be added with our updated weights..

array([[ 2012.17450118,   488.98781631,  2671.71717172,  1242.62847515,
          371.89933007,   524.67930423,   232.65151515,  2038.26482288,
          417.75493001,  2128.37614739,  1125.5667258 ,   791.02715466,
        10000.        ]])

In [92]:
dw = dw/len(Y_batch)
dw

array([[ 2012.17450118,   488.98781631,  2671.71717172,  1242.62847515,
          371.89933007,   524.67930423,   232.65151515,  2038.26482288,
          417.75493001,  2128.37614739,  1125.5667258 ,   791.02715466,
        10000.        ]])

In [93]:
# our final weights will be, weights = weights_initial - (learning rate * dw)
# weights_1 = 0 - (0.000001 * 2012.17450118)

weights = weights - (learning_rate * dw)
weights

array([[-0.00201217, -0.00048899, -0.00267172, -0.00124263, -0.0003719 ,
        -0.00052468, -0.00023265, -0.00203826, -0.00041775, -0.00212838,
        -0.00112557, -0.00079103, -0.01      ]])

In [ ]:
for ind, d in enumerate(distance):
    if max(0, d) == 0:
        di = weights
    else:
        di = weights - (regularization_strength * Y_batch[ind] * X_batch[ind])
    dw += di

In [ ]:
def calculate_cost_gradient(W, X_batch, Y_batch):
    # if only one example is passed (eg. in case of SGD)
    if type(Y_batch) == np.float64:
        Y_batch = np.array([Y_batch])
        X_batch = np.array([X_batch])  # gives multidimensional array

    distance = 1 - (Y_batch * np.dot(X_batch, W))
    dw = np.zeros(len(W))

    for ind, d in enumerate(distance):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (regularization_strength * Y_batch[ind] * X_batch[ind])
        dw += di

    dw = dw/len(Y_batch)  # average
    return dw

In [94]:
weights

array([[-0.00201217, -0.00048899, -0.00267172, -0.00124263, -0.0003719 ,
        -0.00052468, -0.00023265, -0.00203826, -0.00041775, -0.00212838,
        -0.00112557, -0.00079103, -0.01      ]])

In [97]:
features[0], outputs[0]

(array([0.25769361, 0.73336457, 0.53080808, 0.64237574, 0.07818215,
        0.62943491, 0.76717172, 0.62928585, 0.29933115, 0.50868388,
        0.52493594, 0.40968123, 1.        ]),
 -1.0)

In [104]:
N = features.shape[0]  ## Number of samples
distances = 1 - outputs[0] * (np.dot(features[0], weights.T))
distances
#z = np.dot(fetures[0], weights.T)
#z

array([0.98296337])

In [110]:
a = [-1,-2]
a[-1<0] = 0
a

[-1, 0]

In [111]:
distances[distances < 0] = 0  # equivalent to max(0, distance)  as our support vector line crosses in missclassified area..
hinge_loss = regularization_strength * (np.sum(distances) / N)
hinge_loss

21.603590469673385

In [113]:
# calculate cost
cost = 1 / 2 * np.dot(weights, weights.T) + hinge_loss
cost

array([[21.60365256]])

In [116]:
# stoppage criterion
if abs(prev_cost - cost) < cost_threshold * prev_cost:
    print(weights)
else:
    print('weight update needed..')
#prev_cost = cost

weight update needed..


In [ ]:
# so our final 